In [1]:
import scanpy as sc
import anndata
import pl as pl
import tl as tl
import numpy as np
import time as time

# Load Dataset
filelocation = r"/home/felix/Public/datasets/VBh_converted.h5ad"
adata = anndata.read_h5ad(filelocation)

# subsampling
sc.pp.subsample(adata, 0.03)

# normalizing
adata.X = np.arcsinh(adata.X / 10)

# calc knn
sc.pp.neighbors(adata, n_neighbors=20)

In [2]:
def _calc_P(T):
    return (T + T.transpose()) / (2 * T.shape[0])

In [3]:
from scipy.sparse import csr_matrix, spdiags
from scipy.sparse.linalg import eigs

def fresh_calc_T(adata):
    # get connectivities from adata
    c = adata.obsp['connectivities']

    # make sure connectivities are symmetric
    assert(len((c - c.T).data) == 0), "connectivities are not symmetric"

    # row-normalise c to give a transition matrix
    T = c.multiply(csr_matrix(1.0 / np.abs(c).sum(1)))

    # make sure it's correctly row-normalised
    assert(np.allclose(T.sum(1), 1)), "T is not row-normalised"

    # compute the stationary distribution
    #from scipy.sparse.linalg import eigs
    D, V = eigs(T.T, which='LM')
    pi = V[:, 0]

    # make sure pi is entirely real
    assert((pi.imag == 0).all()), "This is not the stationary vector, found imaginary entries"
    pi = pi.real

    # make sure all entries have the same sign
    assert((pi > 0).all() or (pi < 0).all()), "This is not the stationary vector, found positive and negative entries"
    pi /= pi.sum()

    # check pi is normalised correctly
    assert(np.allclose(pi.sum(), 1)), "Pi is not normalized correctly"

    # put the stationary dist into a diag matrix
    Pi = spdiags(pi, 0, pi.shape[0], pi.shape[0])

    # finally, check for reversibility of T
    assert(np.allclose((Pi @ T - T.T @ Pi).data, 0))
    
    # list of landmarks
    
    
    return T
    
t0_new = time.time()
T_new = fresh_calc_T(adata)
t1_new = time.time()

In [4]:
# compared to old method for calculating T
import multiprocessing as mp
from scipy.special import softmax
from scipy.stats import entropy

def _calc_first_T(distances_nn, dim):
    p = mp.Pool(mp.cpu_count())
    probs = p.map(_helper_method_calc_T, [dist.data for dist in distances_nn])
    p.terminate()
    p.join()
    data = []
    for pr in probs:
        data.extend(pr)
    T = csr_matrix((data, distances_nn.indices, distances_nn.indptr), shape=(dim,dim))
    return T

def _helper_method_calc_T(dist):
    d = dist / np.max(dist)
    return softmax((-d ** 2) / _binary_search_sigma(d, len(d)))

def _binary_search_sigma(d, n_neigh):
    # binary search
    sigma = 10  # Start Sigma
    goal = np.log(n_neigh)  # log(k) with k being n_neighbors
    # Do binary search until entropy ~== log(k)
    while True:
        ent = entropy(softmax((-d ** 2) / sigma))
        # check sigma
        if np.isclose(ent, goal):
            return sigma
        if ent > goal:
            sigma *= 0.5
        else:
            sigma /= 0.5


t0_old = time.time()
T_old = _calc_first_T(adata.obsp['distances'], len(adata.X))

t1_old = time.time()



In [5]:
print("--NEW--")
print(f"shape: {np.shape(T_new)}")
print(f"T num entrys: {len(T_new.data)}")
print(f"sum of data: {sum(T_new.data)}")
P_new = _calc_P(T_new)
print(f"sum first row P {sum(sum((P_new.getrow(0)).toarray()))}")
print(f"length first row P {P_new[0].getnnz()}")
print(f"P max {max(P_new.data)}")
print(f"P min {min(P_new.data)}")
print(f"sum first row T {sum(sum((T_new.getrow(0)).toarray()))}")
print(f"T max {max(T_new.data)}")
print(f"T min {min(T_new.data)}")
print(f"time: {t1_new-t0_new}\n")

print("--OLD--")
print(f"shape: {np.shape(T_old)}")
print(f"T num entrys: {len(T_old.data)}")
print(f"sum of data: {sum(T_old.data)}")
P_old = _calc_P(T_old)
print(f"sum first row P {sum(sum((P_old.getrow(0)).toarray()))}")
print(f"length first row P {P_old[0].getnnz()}")
print(f"P max {max(P_old.data)}")
print(f"P min {min(P_old.data)}")
print(f"sum first row T {sum(sum((T_old.getrow(0)).toarray()))}")
print(f"T max {max(T_old.data)}")
print(f"T min {min(T_old.data)}")
print(f"time: {t1_old-t0_old}\n")


--NEW--
shape: (3884, 3884)
T num entrys: 104022
sum of data: 3884.0000001124954
sum first row P 0.00026254706394289895
length first row P 29
P max 5.957213384852228e-05
P min 2.230883490297935e-09
sum first row T 0.9999999644933268
T max 0.2313786894083023
T min 7.353321052505635e-06
time: 0.07435131072998047

--OLD--
shape: (3884, 3884)
T num entrys: 73796
sum of data: 3883.9999999999927
sum first row P 0.00026422856798231334
length first row P 29
P max 1.3861991955299348e-05
P min 6.67671528754798e-06
sum first row T 1.0
T max 0.05407361214597801
T min 0.05178147734568901
time: 0.45916032791137695



In [6]:
n_events = adata.X.shape[0]
init_states = csr_matrix((np.ones(n_events), (range(n_events), range(n_events))))
state = init_states[0]#.toarray()[0]



state_new = state * T_new
state_new = state_new.toarray()[0]
print(sum(state_new))
choice = np.random.choice(range(n_events), 1, p=state_new)
print(choice)


state_old = state * T_old
state_old = state_old.toarray()[0]
print(sum(state_old))
choice = np.random.choice(range(n_events), 1, p=state_old)
print(choice)


for s in enumerate(init_states):
    s_new = init_states[s[0]] * T_new
    s_new = s_new.toarray()[0]
    print(sum(s_new))
    choice = np.random.choice(range(n_events), 1, p=s_new)




0.9999999644933268


ValueError: probabilities do not sum to 1

In [9]:
# OLD LANDMARKS
def _get_landmarks(T, settings):

    n_events = T.shape[0]
    proposals = np.zeros(n_events)  # counts how many times point has been reached
    landmarks = list()  # list of landmarks
    global HELPER_VAR
    HELPER_VAR = {'T': T,
                  'teta': settings['teta'],
                  'beta': settings['beta'],
                  'beta_thresh': settings['beta_thresh'],
                  'n_events': n_events}
    init_states = csr_matrix((np.ones(n_events), (range(n_events), range(n_events))))
    p = mp.Pool(mp.cpu_count())
    hit_list = p.map(_helper_method_get_landmarks, [state for state in init_states])
    p.terminate()
    p.join()
    # evaluate results
    for state_hits in hit_list:  # for every states hit_list
        for h in state_hits:  # for every hit in some states hit_list
            proposals[h[0]] += h[1]

    # collect landmarks
    min_beta = settings['beta'] * settings['beta_thresh']
    for prop in enumerate(proposals):
        # if event has been hit min_beta times, it counts as landmark
        if prop[1] > min_beta:
            landmarks.append(prop[0])
    return landmarks

def _helper_method_get_landmarks(state):
    for i in range(HELPER_VAR['teta']):
        state *= HELPER_VAR['T']
    destinations = np.random.choice(range(HELPER_VAR['n_events']), HELPER_VAR['beta'], p=state.toarray()[0])
    hits = np.zeros((HELPER_VAR['n_events']))
    for d in destinations:
        hits[d] += 1
    return [(h[0], h[1]) for h in enumerate(hits) if h[1] > 0]

##################


# NEW LANDMARKS
def fresh_calc_lm(T):

    # make sure it's correctly row-normalised
    assert(np.allclose(T.sum(1), 1)), "T is not row-normalised"

    # compute the stationary distribution
    #from scipy.sparse.linalg import eigs
    D, V = eigs(T.T, which='LM')
    pi = V[:, 0]

    # make sure pi is entirely real
    #assert((pi.imag == 0).all()), "This is not the stationary vector, found imaginary entries"
    pi = pi.real

    # make sure all entries have the same sign
    #assert((pi > 0).all() or (pi < 0).all()), "This is not the stationary vector, found positive and negative entries"
    pi /= pi.sum()

    # check pi is normalised correctly
    assert(np.allclose(pi.sum(), 1)), "Pi is not normalized correctly"

    # put the stationary dist into a diag matrix
    Pi = spdiags(pi, 0, pi.shape[0], pi.shape[0])

    # finally, check for reversibility of T
    #assert(np.allclose((Pi @ T - T.T @ Pi).data, 0))
    
    # new: get rid of negative stuff
    pi += abs(min(pi))
    pi /= sum(pi)
    
    # list of landmarks
    return pi, pi.argsort()

##################

settings = {'T': T_old,
              'teta': 50,
              'beta': 100,
              'beta_thresh': 1.5,
              'n_events': T_old.shape[0]}
print(f"n_events: {settings['n_events']}")

lm_old = _get_landmarks(T_old, settings)
print('lm_old')
print(lm_old[:10])
print(len(lm_old))

pi, lm_new = fresh_calc_lm(T_old)
print('lm_new')
print(lm_new)
print(pi)
print(len(lm_new))

n_events: 3884
lm_old
[8, 10, 11, 13, 15, 20, 22, 28, 29, 30]
867
lm_new
[3607 2281  552 ... 2798  190 3686]
[0.00025746 0.00025739 0.0002557  ... 0.00026743 0.0002596  0.00025744]
3884


In [8]:
x = lm_new

x += abs(min(x))
x = x / sum(x)

print(min(x))
print(max(x))
print(sum(x))
print(x)


0.0
0.0005149330587023687
0.9999999999999993
[1.65632601e-04 4.73292837e-04 2.72518011e-04 ... 7.32019182e-05
 3.02488361e-04 4.78332100e-04]
